In [1]:
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from IPython.display import clear_output

In [2]:
df_kodex = pd.read_excel("data_kodex_kospi.xlsx")
df_kodex

,DATE,TIME,DATETIME,KDX_OPN,KDX_CLS,KDX_HIGH,KDX_LOW,KDX_VOL
0,2021-09-24,10,2021092410,31510,31535,31595,31500,1963
1,2021-09-24,11,2021092411,31520,31435,31570,31435,1792
2,2021-09-24,12,2021092412,31440,31440,31455,31395,778
3,2021-09-24,13,2021092413,31440,31425,31445,31355,854
4,2021-09-24,14,2021092414,31420,31425,31435,31365,1110
...,...,...,...,...,...,...,...,...
995,2022-04-22,12,2022042212,27665,27630,27670,27615,69818
996,2022-04-22,13,2022042213,27625,27660,27680,27620,71542
997,2022-04-22,14,2022042214,27660,27690,27700,27645,67847
998,2022-04-22,15,2022042215,27690,27725,27740,27690,79416


In [3]:
df_kodex_inv = pd.read_excel("data_kodex_kospi_inverse.xlsx")
df_kodex_inv

,DATE,TIME,DATETIME,KDI_OPN,KDI_CLS,KDI_HIGH,KDI_LOW,KDI_VOL
0,2021-09-24,11,2021092411,3915,3925,3925,3900,5804180
1,2021-09-24,12,2021092412,3925,3920,3930,3915,4590549
2,2021-09-24,13,2021092413,3920,3925,3930,3915,2754191
3,2021-09-24,14,2021092414,3925,3920,3930,3920,1777032
4,2021-09-24,15,2021092415,3920,3920,3930,3920,2311850
...,...,...,...,...,...,...,...,...
995,2022-04-22,12,2022042212,4405,4410,4410,4395,2076584
996,2022-04-22,13,2022042213,4405,4400,4410,4400,969359
997,2022-04-22,14,2022042214,4405,4395,4410,4395,1958333
998,2022-04-22,15,2022042215,4395,4395,4400,4385,2278272


In [4]:
df = df_kodex.merge(df_kodex_inv, how="left", on=["DATE", "TIME", "DATETIME"])
df.dropna(inplace=True)
df.sort_values("DATETIME", inplace=True)
df.reset_index(drop=True, inplace=True)
df["DATETIME"] = df["DATETIME"].astype(str)

df_train = df[6:698]
df_test = df[698:]
df_train

,DATE,TIME,DATETIME,KDX_OPN,KDX_CLS,KDX_HIGH,KDX_LOW,KDX_VOL,KDI_OPN,KDI_CLS,KDI_HIGH,KDI_LOW,KDI_VOL
6,2021-09-27,10,2021092710,31425,31500,31500,31350,5598,3930.0,3910.0,3935.0,3905.0,8735966.0
7,2021-09-27,11,2021092711,31500,31595,31600,31480,2375,3905.0,3900.0,3915.0,3890.0,4966691.0
8,2021-09-27,12,2021092712,31555,31620,31630,31555,1924,3900.0,3900.0,3905.0,3895.0,1028966.0
9,2021-09-27,13,2021092713,31620,31470,31620,31460,13118,3900.0,3910.0,3915.0,3895.0,3728783.0
10,2021-09-27,14,2021092714,31515,31465,31515,31440,3366,3915.0,3920.0,3925.0,3910.0,2526952.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
693,2022-02-18,12,2022021812,28080,27990,28080,27960,11550,4260.0,4270.0,4275.0,4255.0,2737173.0
694,2022-02-18,13,2022021813,27980,28045,28100,27955,27853,4265.0,4265.0,4275.0,4245.0,3263025.0
695,2022-02-18,14,2022021814,28020,28040,28075,28000,23193,4265.0,4255.0,4265.0,4255.0,2776279.0
696,2022-02-18,15,2022021815,28040,28070,28095,28025,23095,4255.0,4255.0,4265.0,4250.0,2730596.0


In [5]:
class modelTrainer(): #일봉 기준 클래스로, 추후 시간봉으로 트레이닝할 때는 내용수정이 필요함

    def __init__(self, df):
        self.df = df
        self.state_list = None
        self.action_names = ["KDX매수", "KDX매도", "홀드", "KDI매수", "KDI매도"]
        self.last_buy_price_kdx = 0
        self.last_buy_price_kdi = 0
        self.model_pred = None
        self.model_q = None

    def makeStateList(self, before_range=60):
        state_list = []
        for idx in range(len(self.df)):
            if idx > before_range-1:
                this_mat = self.df.iloc[idx-before_range:idx, 2:].to_numpy() #DATETIME부터 마지막 열까지
                for col_idx in range(1, this_mat.shape[1]):
                    this_arr = this_mat[:, col_idx]
                    max_val = max(this_arr)
                    min_val = min(this_arr)
                    for row_idx, val in enumerate(this_arr):
                        new_val = (val - min_val) / (max_val - min_val)
                        this_mat[row_idx, col_idx] = new_val
                state_list.append(this_mat)

        self.state_list = np.array(state_list)

    def initModel(self):
        self.model_pred = tf.keras.Sequential([
            tf.keras.layers.Input((self.state_list.shape[1]*(self.state_list.shape[2]-1))+3),
            tf.keras.layers.Dense(units=128, activation="relu"),
            tf.keras.layers.Dense(units=128, activation="relu"),
            tf.keras.layers.Dense(units=128, activation="relu"),
            tf.keras.layers.Dense(units=128, activation="relu"),
            tf.keras.layers.Dense(units=128, activation="relu"),
            tf.keras.layers.Dense(units=5)
        ])
        self.model_pred.compile(optimizer=tf.keras.optimizers.Adam(), 
                    loss="mse", 
                    metrics=["mae", "mse"])

        self.model_pred.summary()
        self.model_pred.save("model_pred.h5")
        self.model_q = tf.keras.models.load_model("model_pred.h5")        

    def train(self):
        if self.model_pred is None or self.model_q is None:
            print("모델을 먼저 생성해 주세요!")
            return False
        self.capital = 1000 * 10000
        self.dis = .9 #discounted reward를 위해
        self.profit_rate_goal = 1.5 #terminal 목표 수익율 설정
        profit_rate_under_limit = 0.999 #terminal 하한선 설정
        update_period = 120 #스테이지 120회당 1번씩 pred 모델의 웨이트를 업데이트 한다.
        copy_period = 180 #스테이지 180번당 q모델을 pred 모델에서 복제한다.
        cnt_for_update = 0
        cnt_for_copy = 0
        episodes = 3000
        batch_buffer = []
        reward_list = []
        result = []
        for episode in range(1, episodes+1):
            clear_output(wait=True)
            
            #변수 초기화
            capital = self.capital #사전에 설정한 capital로 초기화
            self.remain_cash = capital
            self.remain_cash_kdx = 0
            self.remain_cash_kdi = 0
            self.remain_stock_kdx = 0
            self.remain_stock_kdi = 0

            done = False

            for state_idx, state in enumerate(self.state_list):
                self.now_state = state
                cnt_for_update += 1
                cnt_for_copy += 1

                #update_period 횟수가 되면 pred 모델을 학습시키고 저장한다.
                if cnt_for_update+1 == update_period:
                    batch_buffer = np.array(batch_buffer, dtype=np.float64)
                    batch_buffer = batch_buffer.reshape(batch_buffer.shape[0], batch_buffer.shape[2])

                    reward_list = np.array(reward_list, dtype=np.float64)

                    self.model_pred.fit(batch_buffer, reward_list, verbose=0, batch_size=32)
                    self.model_pred.save("model_pred.h5")
                    batch_buffer = [] #업데이트 한 후 미니배치(x)를 초기화한다.
                    reward_list = [] #업데이트 한 후 보상값(y)을 초기화한다.
                    cnt_for_update = 0

                #특정 시점(copy_period)가 되면 q모델을 pred모델로부터 복제해 온다.
                if cnt_for_copy+1 == copy_period:
                    self.model_q = tf.keras.models.load_model("model_pred.h5")
                    cnt_for_copy = 0

                #현재 스테이트의 정보를 불러온다. (60행 * 10열)
                state_data = state[:, 1:]

                #flatten => 600행
                state_data = state_data.reshape(state_data.shape[0]*state_data.shape[1])

                #현재 수익율(=잔고), kdx, kdi 잔고(비율)을 마지막 요소로 입력 => 603행
                remain_cash_sign = 0
                remain_kdx_sign = 0
                remain_kdi_sign = 0
                if self.remain_stock_kdx > 0:
                    remain_kdx_sign = 1
                elif self.remain_stock_kdi > 0:
                    remain_kdi_sign = 1
                else:
                    remain_cash_sign = 1
                state_data = np.append(state_data, remain_cash_sign)
                state_data = np.append(state_data, remain_kdx_sign)
                state_data = np.append(state_data, remain_kdi_sign)
                state_data = np.expand_dims(state_data, 0)
                batch_buffer.append(state_data) #pred 학습을 위해 먼저 미니배치 버퍼에 x를 넣어놓는다.

                #Q 모델에게 물어본다.
                pred = self.model_q.predict(state_data)
                action_selected = np.argmax(pred[0])
                action_name_org = self.action_names[action_selected] # 화면 출력용

                #exploration을 위해 에피소드 초기에는 랜덤하게 액션을 선택하게 한다.
                e = 1. / ((episode / 100) + 1)
                if np.random.rand(1) < e:
                    action_selected = random.randrange(0, 2)
                action_name = self.action_names[action_selected] # 화면 출력용

                #액션별 보상을 계산한다.
                reward, action_to_run = self._calcReward(self, action_selected)

                #보상을 reward_list에 반영한다. (y값이 될 것)
                reward_list.append(reward)

                #다음 스테이트 준비를 위해 액션을 취하고 액션별 수익을 갱신한다. (s+1 시가로 매수 / 매도 / 유지)
                self._runActionAndUpdateRemain(action_to_run)














                if state_idx == len(state_list)-2: #마지막 스테이트라면 (실제로는 마지막 스테이트 이전이지만)
                    done = True
                else:
                    #수익율이 상한선에 다다르면 reward는 1, 하한선에 다달았다면 reward를 0으로 주고 에피소드를 종료한다.
                    if snd_profit_rate >= self.profit_rate_goal:
                        reward = 1
                        done = True
                    elif snd_profit_rate <= profit_rate_under_limit:
                        reward = 0
                        done = True

                print("\n\n[EPISODE{}, S{} : {} → {}] {} {}".format(
                    episode, state_idx, action_name_org, action_name, pred[0], reward), end="\r")
   
                print("\n잔고 : {:,d}원".format(int(self.remain_cash + self.remain_cash_kdx + self.remain_cash_kdi), end="\r"))

                if done is True:
                    result.append({
                        "EPISODE" : episode,
                        "MAX. STATE" : state_idx,
                        "잔고" : remain_cash,
                        "수익" : remain_cash - capital,
                        "수익율" : remain_cash / capital
                    })
                    break

            df_result = pd.DataFrame(result)
            df_result.to_excel("result.xlsx", index=False)

    def _runActionAndUpdateRemain(self, action_to_run):
        if action_to_run == 0: #KDX 매수라면
            self.remain_cash
            self.remain_cash_kdx
            self.remain_cash_kdi
            self.remain_stock_kdx
            self.remain_stock_kdi
        elif action_to_run == 1: #KDX 매도라면
            self.remain_cash
            self.remain_cash_kdx
            self.remain_cash_kdi
            self.remain_stock_kdx
            self.remain_stock_kdi
        elif action_to_run == 2: #유지라면
            self.remain_cash
            self.remain_cash_kdx
            self.remain_cash_kdi
            self.remain_stock_kdx
            self.remain_stock_kdi                    
        elif action_to_run == 3: #KDI 매수라면
            self.remain_cash
            self.remain_cash_kdx
            self.remain_cash_kdi
            self.remain_stock_kdx
            self.remain_stock_kdi
        elif action_to_run == 4: #KDI 매도라면
            self.remain_cash
            self.remain_cash_kdx
            self.remain_cash_kdi
            self.remain_stock_kdx
            self.remain_stock_kdi
    
    # 보상설계가 핵심!
    def _calcProfit(self, alpha, beta): #alpha : kdx or kdi, beta : buy or sell or hold / 수익의 기준은 1주 거래 후 남은 금액으로 함
        # s+1, s+2의 데이터(df)를 불러온다
        s1_df_idx = self.df.loc[(df["DATETIME"] == self.now_state[-1, 0])].index[0] + 1
        s2_df_idx = self.df.loc[(df["DATETIME"] == self.now_state[-1, 0])].index[0] + 2
        s1_day_df = self.df.loc[s1_df_idx]
        s2_day_df = self.df.loc[s2_df_idx]

        if alpha == "KDX":
            col_prefix = "KODEX_"
            remain_of_stock = self.remain_stock_kdx
        elif alpha == "KDI":
            col_prefix = "KODEX_INV_"
            remain_of_stock = self.remain_stock_kdi

        # 매수/매도 시점은 항상 시가
        if beta == "BUY": # s+1의 시가로 산 주식을 s+2 스테이트의 시가로 팔았다고 가정하여 수익 계산
            s1_profit = s2_day_df[col_prefix+"OPN"] - s1_day_df[col_prefix+"OPN"]
        elif beta == "SELL":
            # 현재 들고 있는 KDX(or KDI)의 s+1 시가 기준으로 가치 계산
            s1_profit = remain_of_stock * s1_day_df[col_prefix+"OPN"]
        elif beta == "HOLD":
            # 현재 들고 있는 KDX(or KDI)의 s+2 시가 기준으로 가치 계산
            s1_profit = remain_of_stock * s2_day_df[col_prefix+"OPN"]

        # s+2의 max reward는?
        # 현재 들고 있는 현금 혹은 주식가치만큼의 현금이 있다고 가정한다.
        if self.remain_cash_kdx > 0:
            capital_assume = self.remain_cash_kdx
        elif self.remain_cash_kdi > 0:
            capital_assume = self.remain_cash_kdi
        else:
            capital_assume = self.remain_cash

        # case1 : s+1 KDX 시가에서 s+2 KDX 시가만큼의 차이만큼 수익이 있다고 계산한다.
        kdx_stock_cnt = capital_assume // s1_day_df["KDX_OPN"] #살수 있는 주식 수
        ca_rest = capital_assume % s1_day_df["KDX_OPN"]
        s2_profit_1 = (kdx_stock_cnt * s2_day_df["KDX_OPN"]) + ca_rest

        # case : s+1 KDI 시가에서 s+2 KDI 시가만큼의 차이만큼 수익이 있다고 계산한다.
        kdi_stock_cnt = capital_assume // s1_day_df["KDI_OPN"] #살수 있는 주식 수
        ca_rest = capital_assume % s1_day_df["KDI_OPN"]
        s2_profit_2 = (kdi_stock_cnt * s2_day_df["KDI_OPN"]) + ca_rest

        s2_profit = s2_profit_1
        if s2_profit_2 > s2_profit_1:
            s2_profit = s2_profit_2

        max_profit = s1_profit + (self.dis * (capital_assume + s2_profit)) # --> 핵심이 될것!!!!!!!!!!!!!!!!!
        reward = max_profit / (self.capital * self.profit_rate_goal) #reward 수치로 변환한다.

        return s1_profit, max_profit, reward

    def _calcReward(self, action_selected):
        if action_selected == 0:
            if self.remain_cash > 0:
                _, _, a0_reward = self._calcProfit("KDX", "BUY")
                _, _, a3_reward = self._calcProfit("KDI", "BUY")
                reward_result = [a0_reward, -1, -1, a3_reward, -1]
                action_to_run = 0
                self.remain_cash
                self.remain_cash_kdx
                self.remain_cash_kdi
                self.remain_stock_kdx
                self.remain_stock_kdi
            elif self.remain_stock_kdx > 0:
                _, _, a1_reward = self._calcProfit("KDX", "SELL")
                _, _, a2_reward = self._calcProfit("KDX", "HOLD")
                reward_result = [-1, a1_reward, a2_reward, -1, -1]
                action_to_run = 2
            elif self.remain_stock_kdi > 0:
                _, _, a4_reward = self._calcProfit("KDI", "SELL")
                _, _, a2_reward = self._calcProfit("KDI", "HOLD")
                reward_result = [-1, -1, a2_reward, -1, a4_reward]
                action_to_run = 2

        elif action_selected == 1:
            if self.remain_stock_kdx > 0:
                _, _, a1_reward = self._calcProfit("KDX", "SELL")
                _, _, a2_reward = self._calcProfit("KDX", "HOLD")
                reward_result = [-1, a1_reward, a2_reward, -1, -1]
                action_to_run = 1
            elif self.remain_stock_kdi > 0:
                _, _, a4_reward = self._calcProfit("KDI", "SELL")
                _, _, a2_reward = self._calcProfit("KDI", "HOLD")
                reward_result = [-1, -1, a2_reward, -1, a4_reward]
                action_to_run = 2
            elif self.remain_cash > 0:
                _, _, a0_reward = self._calcProfit("KDX", "BUY")
                _, _, a3_reward = self._calcProfit("KDI", "BUY")
                reward_result = [a0_reward, -1, -1, a3_reward, -1]
                action_to_run = 2
                
        elif action_selected == 2:
            if self.remain_cash > 0:
                _, _, a0_reward = self._calcProfit("KDX", "BUY")
                _, _, a3_reward = self._calcProfit("KDI", "BUY")
                reward_result = [a0_reward, -1, -1, a3_reward, -1]
                action_to_run = 2
            elif self.remain_stock_kdx > 0:
                _, _, a1_reward = self._calcProfit("KDX", "SELL")
                _, _, a2_reward = self._calcProfit("KDX", "HOLD")
                reward_result = [-1, a1_reward, a2_reward, -1, -1]
                action_to_run = 2
            elif self.remain_stock_kdi > 0:
                _, _, a4_reward = self._calcProfit("KDI", "SELL")
                _, _, a2_reward = self._calcProfit("KDI", "HOLD")
                reward_result = [-1, -1, a2_reward, -1, a4_reward]
                action_to_run = 2

        elif action_selected == 3:
            if self.remain_cash > 0:
                _, _, a0_reward = self._calcProfit("KDX", "BUY")
                _, _, a3_reward = self._calcProfit("KDI", "BUY")
                reward_result = [a0_reward, -1, -1, a3_reward, -1]
                action_to_run = 3
            elif self.remain_stock_kdx > 0:
                _, _, a1_reward = self._calcProfit("KDX", "SELL")
                _, _, a2_reward = self._calcProfit("KDX", "HOLD")
                reward_result = [-1, a1_reward, a2_reward, -1, -1]
                action_to_run = 2
            elif self.remain_stock_kdi > 0:
                _, _, a4_reward = self._calcProfit("KDI", "SELL")
                _, _, a2_reward = self._calcProfit("KDI", "HOLD")
                reward_result = [-1, -1, a2_reward, -1, a4_reward]
                action_to_run = 2

        elif action_selected == 4:
            if self.remain_stock_kdi > 0:
                _, _, a4_profit = self._calcProfit("KDI", "SELL")
                _, _, a2_profit = self._calcProfit("KDI", "HOLD")
                reward_result = [-1, -1, a2_profit, -1, a4_profit]
                action_to_run = 4
            elif self.remain_stock_kdx > 0:
                _, _, a1_profit = self._calcProfit("KDX", "SELL")
                _, _, a2_profit = self._calcProfit("KDX", "HOLD")
                reward_result = [-1, a1_profit, a2_profit, -1, -1]
                action_to_run = 2
            elif self.remain_cash > 0:
                _, _, a0_profit = self._calcProfit("KDX", "BUY")
                _, _, a3_profit = self._calcProfit("KDI", "BUY")
                reward_result = [a0_profit, -1, -1, a3_profit, -1]
                action_to_run = 2
        return reward_result, action_to_run

trainer = modelTrainer(df_train)
trainer.makeStateList()

In [10]:
trainer.state_list[0][0, 0]

'2021092710'

# 추론 / 테스트 데이터에 적용

In [ ]:
#특정 년도 데이터로 추론해 본다.
year = "2020"
df_test = df_kospi.merge(df_kodex, how="right", on="date")
df_test = df_test.merge(df_kodex_inv, how="left", on="date")
df_test.dropna(inplace=True)
df_test.sort_values("date", inplace=True)
df_test.reset_index(drop=True, inplace=True)

df_test["date"] = pd.to_datetime(df_test["date"])
df_test = df_test[(df_test["date"] >= "{}-01-01".format(year)) & (df_test["date"] <= "{}-12-31".format(year))]
df_test.reset_index(drop=True, inplace=True)
df_test

In [ ]:
state_list = []
for idx in range(len(df_test)):
    if idx > 59:
        this_mat = df_test.iloc[idx-60:idx, 0:5].to_numpy()
        for col_idx in range(1, this_mat.shape[1]):
            this_arr = this_mat[:, col_idx]
            max_val = max(this_arr)
            min_val = min(this_arr)
            for row_idx, val in enumerate(this_arr):
                new_val = (val - min_val) / (max_val - min_val)
                this_mat[row_idx, col_idx] = new_val
        state_list.append(this_mat)

state_list = np.array(state_list)
state_list.shape

#6개열 : date, KOSPI, KOSPI_START, KOSPI_HIGH, KOSPI_LOW, CAPITAL

In [ ]:
model = tf.keras.models.load_model("model_pred_1.2.h5")
action_names = ["KDX매수", "KDI매수"]
capital = 1000 * 10000
remain_cash = capital
remain_kdx = 0
remain_kdi = 0
result = []
for state_idx, state in enumerate(state_list):
    #현재 스테이트의 정보를 불러온다. (60행 * 4열)
    state_data = np.array(state[:, 1:], dtype=np.float64)

    #flatten => 240행
    state_data = state_data.reshape(state_data.shape[0]*state_data.shape[1])

    #현재 수익율(=잔고), kdx, kdi 잔고(비율)을 마지막 요소로 입력 => 241행 / 추후 액션 갯수를 더 늘리기 위해 작업해 놓는것... (매도/홀딩 액션 감안)
    state_data = np.append(state_data, remain_cash / capital)
    state_data = np.expand_dims(state_data, 0)

    #Q 모델에게 물어본다.
    pred = model.predict(state_data)
    action_selected = np.argmax(pred[0])
    action_name = action_names[action_selected]
    # print("STATE {} : {}".format(state_idx, action_name))

    #다음날의 데이터를 불러온다.
    snd_state_idx = df_test.loc[(df_test["date"] == state[-1, 0])].index[0] + 1
    sec_state_data = df_test.loc[snd_state_idx]

    #액션별 수익을 계산한다.
    buy_cnt_kdx = np.floor(remain_cash / sec_state_data["KODEX_OPN"])
    profit_kdx = (buy_cnt_kdx * (sec_state_data["KODEX_CLS"] - sec_state_data["KODEX_OPN"]))
    remain_cash_kdx = remain_cash + profit_kdx

    buy_cnt_kdi = np.floor(remain_cash / sec_state_data["KODEX_INV_OPN"])
    profit_kdi = (buy_cnt_kdi * (sec_state_data["KODEX_INV_CLS"] - sec_state_data["KODEX_INV_OPN"]))
    remain_cash_kdi = remain_cash + profit_kdi

    #액션을 취하고 액션별 수익을 반영한다. (다음날 시작 가격으로 매수 혹은 매도)
    if action_name == "KDX매수":
        profit = profit_kdx
        remain_cash = remain_cash_kdx
    elif action_name == "KDI매수":
        profit = profit_kdi
        remain_cash = remain_cash_kdi

    #수익율을 계산해 본다.
    snd_profit_rate = remain_cash / capital

    result.append({
        "STATE" : state_idx,
        "잔고" : remain_cash,
        "수익" : remain_cash - capital,
        "수익율" : remain_cash / capital
    })

df_result = pd.DataFrame(result)
df_result.to_excel("result_test.xlsx", index=False)
df_result

In [ ]:
df_result["잔고"].plot()